In [1]:
import numpy as np

import matplotlib.pylab as plt
import matplotlib.animation as animation

import warnings
from itertools import product as cartesian_product

from pprint import pprint
import pickle

from tqdm import tqdm

import IPython
from IPython.display import HTML

### Configuração inicial das bibliotecas importadas

In [2]:
warnings.filterwarnings("ignore")
plt.rcParams['axes.titlepad'] = 30

### Configurações do jogo - Parte 1

In [3]:
MAPA_ELEMENTOS = {
    'vazio': 0,
    'parede': 1,
    'cabeça': 2,
    'corpo': 3,
    'comida': 4,
}

MAPA_ELEMENTOS_REVERSO = {
    0: 'vazio',
    1: 'parede',
    2: 'cabeça',
    3: 'corpo',
    4: 'comida',
}

MAPA_CORES = {
    0: np.array((65, 193, 79)),
    1: np.array((75, 112, 47)),
    2: np.array((27, 70, 158)),
    3: np.array((70, 121, 241)),
    4: np.array((233, 71, 29)),
}

## Funcionalidades do Jogo

### Gerar Cenário

Vamos criar aqui uma função que vai criar um cénario de jogo, assim que receber o tamanho de Linhas e Colunas.

O mapa vai ser criado com mais 2 linhas e mais 2 colunas, para serem as colunas, ou seja, definimos apenas o espaço Jogável do mapa

In [4]:
def criar_cenario(linhas=200, colunas=200):
    mapa = np.zeros(shape=(linhas+2, colunas+2)).astype(np.uint8)

    # preenchendo as bordas com parede
    for cada_linha in range(linhas+2): #preenchendo as colunas 0 e N-1
        mapa[cada_linha, 0] = MAPA_ELEMENTOS['parede']
        mapa[cada_linha, colunas+1] = MAPA_ELEMENTOS['parede']
    
    for cada_coluna in range(colunas+2): # preenchendo as linhas 0 e M-1
        mapa[0, cada_coluna] = MAPA_ELEMENTOS['parede']
        mapa[linhas+1, cada_coluna] = MAPA_ELEMENTOS['parede']
    
    return mapa

### Substituir Cores

Como precisamos visualizar o jogo de forma bonita, vamos substituir cada bloco no mapa por um vetor contendo a cor do bloco desejado, para isso que criamos a variável MAPA_CORES

In [5]:
def substituir_cores(mapa, mapa_cores=MAPA_CORES):
    linhas, colunas = mapa.shape
    matriz_mapa_cores = np.zeros(shape=(linhas, colunas, 3)).astype(np.uint8)

    # Percorre cada bloco no mapa
    for cada_linha in range(linhas):
        for cada_coluna in range(colunas):
            # substitui as cores
            matriz_mapa_cores[cada_linha, cada_coluna] = mapa_cores[mapa[cada_linha, cada_coluna]]
        
    return matriz_mapa_cores

### Renderizar Cena

Esse método não vai ser crucial para o Jogo, mas vamos utiliza-lo bastante para gerar "prints" do jogo e exibi-las:

In [6]:
def renderizar_cena_jogo(matriz_mapa, mapa_cores=MAPA_CORES, figura_tamanho=[]):
    x, y = matriz_mapa.shape[::-1]

    # Define um tamanho para o gráfico em polegadas
    if figura_tamanho:
        plt.figure(figsize=figura_tamanho)
    else:
        plt.figure(figsize=np.array([x, y]) / 5)
    
    # Gera o mapa com pixels RGB
    matriz_mapa_cores = substituir_cores(matriz_mapa, MAPA_CORES)

    # Exibe o gráfico
    plt.imshow(matriz_mapa_cores, aspect='auto')
    plt.axis('off')
    plt.show()

### Gerar comida

Por meio dessa função vamos criar pontos no mapa que serão as comidas.

Vamos criar comida apenas onde houver blocos vazios.

In [7]:
def gerar_comida(matriz_mapa, posicao_inicial=None):
    # Se não estiver definida uma posição inicial
    if not posicao_inicial:
        linhas, colunas = matriz_mapa.shape

        # Geramos pontos aleatórios dentre a área acessível do mapa até achar algo vago
        x = np.random.randint(low=1, high=colunas-2)
        y = np.random.randint(low=1, high=linhas-2)

        while matriz_mapa[y, x] != MAPA_ELEMENTOS['vazio']:
            x = np.random.randint(low=1, high=colunas-2)
            y = np.random.randint(low=1, high=linhas-2)
    
    else:
        # Caso contrário apenas usamos as posições desejadas
        x, y = posicao_inicial
    
    # Invertemos x, y pela diferença de Matriz e Plano cartesiano
    matriz_mapa[y, x] = MAPA_ELEMENTOS['comida']

    return (x, y)

## Classes do Jogo

### Cobra

Agora vamos definir a Cobrinha do nosso jogo, como uma estrutura um pouco mais consistente.

Como a cobra tem várias funções e tudo mais, vamos gerar uma classe para armazenar tudo em um lugar só.

In [8]:
from email.mime import base
from hashlib import new


class Cobra():
    # Inicialização da classe, ela recebe o mapa, uma posivel posição
    # Inicial e um possível tamanho inicial
    def __init__(self, mapa, posiciao_inicial=None, tamanho_inicial=1) -> None:
        
        self.mapa = mapa
        self.size = tamanho_inicial
        self.movimentos_rastro = []
        self.jogadas = 0

        # Se nenhuma posição inicial for passada crie uma aleatoriamente
        # dentro do espaço disponível
        if not posiciao_inicial:
            linhas, colunas = mapa.shape
            self.x = np.random.randint(low=1, high=colunas-2)
            self.y = np.random.randint(low=1, high=linhas-2)

            while self.mapa[self.y, self.x] != MAPA_ELEMENTOS['vazio']:
                self.x = np.random.randint(low=1, high=colunas-2)
                self.y = np.random.randint(low=1, high=linhas-2)

        else:
            self.x, self.y = posiciao_inicial

        # Para simular a ideia de corpo da cobra, podemos criar um array
        # que vai conter as últimas posições do corpo da cobra
        # sempre o último elemento desse array vai ser a cabeça
        self.movimentos_rastro.append([self.x, self.y]) 
        self.mapa[self.y, self.x] = MAPA_ELEMENTOS['cabeça']

        # Caso nos precisarmos de criar uma cobra que já começa com
        # um corpo grande, podemos tentar criar o corpo de forma reta
        # até encontrar algum bloqueio e mudar a direção do crescimento.
        if tamanho_inicial > 1:
            base_x, base_y = self.x, self.y
            for i in range(1, tamanho_inicial):
                esquerda = (base_y, base_x - 1)
                acima = (base_y - 1, base_x)
                direita = (base_y, base_x + 1)
                abaixo = (base_y + 1, base_x)
                encontrou = 0
                posicoes_possiveis = [esquerda, acima, direita, abaixo]
                for possivel_y, possivel_x in posicoes_possiveis:
                    if self.mapa[possivel_y, possivel_x] == MAPA_ELEMENTOS['vazio']:
                        self.movimentos_rastro.append([possivel_x, possivel_y])    
                        self.mapa[possivel_y, possivel_x] = MAPA_ELEMENTOS['corpo']
                        base_x, base_y = possivel_x, possivel_y
                        encontrou = 1
                        break
                if encontrou == 0:
                    self.size = i 
            self.movimentos_rastro = self.movimentos_rastro[::-1]

    def obter_posicao_cabeca(self):
        return (self.x, self.y)

    def obter_tamanho(self):
        return self.size

    def obter_posicao_pescoco(self):
        resposta = None

        if len(self.movimentos_rastro) >= 2:
            x_pescoco, y_pescoco = self.movimentos_rastro[-2]

            if x_pescoco == self.x and y_pescoco == self.y - 1:
                resposta = 'ACIMA'       
            elif x_pescoco == self.x and y_pescoco == self.y + 1:
                resposta = 'ABAIXO'
            elif x_pescoco == self.x - 1 and y_pescoco == self.y:
                resposta = 'ESQ'
            elif x_pescoco == self.x + 1 and y_pescoco == self.y:
                resposta = 'DIR'

        return resposta
    
    # Com essa função vai ser possível obter informações do ambiente
    # que está visível para a cobra

    def obter_area_cabeca(self):
        esquerda = self.mapa[self.y, self.x - 1]
        acima = self.mapa[self.y - 1, self.x]
        direita = self.mapa[self.y, self.x + 1]
        abaixo = self.mapa[self.y + 1, self.x]

        return (esquerda, acima, direita, abaixo)
    
    def eliminar_rastro(self):
        tamanho_rastro = len(self.movimentos_rastro)
        if (tamanho_rastro > self.size):
            for i in range(0, tamanho_rastro - self.size):
                x_antigo, y_antigo = self.movimentos_rastro[0]
                self.mapa[y_antigo, x_antigo] = MAPA_ELEMENTOS['vazio']
                self.movimentos_rastro.pop(0)
    
    # Executa a movimentação da cobrinha
    def movimento(self, direcao: str):
        if (direcao == 'ESQ'):
            new_x = self.x - 1
            new_y = self.y
        elif (direcao == 'DIR'):
            new_x = self.x + 1
            new_y = self.y
        elif (direcao == 'ACIMA'):
            new_x = self.x
            new_y = self.y - 1
        elif (direcao == 'ABAIXO'):
            new_x = self.x
            new_y = self.y + 1

        bloco_alvo = self.mapa[new_y, new_x]
        if (bloco_alvo == MAPA_ELEMENTOS['comida']):
            self.size += 1
        self.mapa[self.y, self.x] = MAPA_ELEMENTOS['corpo']
        self.mapa[new_y, new_x] = MAPA_ELEMENTOS['cabeça']
        self.x, self.y = new_x, new_y

        # Elimina os rastros
        self.movimentos_rastro.append([new_x, new_y])
        self.eliminar_rastro()
        self.jogadas += 1

        return bloco_alvo

### Q-Learn

Todo o processo de aprendizagem e tomada de decisão vai ficar por responsabilidade da classe de Qlearn

Nessa vai ficar contida todo o processo de aprendizagem

In [10]:
from socket import SIO_KEEPALIVE_VALS
from tkinter import VERTICAL


class QLearn:

    def __init__(self, epsilon=0.2, alpha=0.05, gamma=0.001) -> None:

        self.qtable = None
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma

        # Possíveis ações a serem tomadas
        self.acoes = {
            0: 'ESQ',
            1: 'ACIMA',
            2: 'DIR',
            3: 'ABAIXO',
        }

        # Possíveis ações a serem tomadas Invertida
        self.acoes_rev = {
            'ESQ': 0,
            'ACIMA': 1,
            'DIR': 2,
            'ABAIXO': 3,
        }    
        self.inicializar_qtable()

    # Inicializa a Qtable com valores zerados
    def inicializar_qtable(self):
        nova_qtable = {}
        horizontal_comida = [-1, 0, 1]
        vertical_comida = [-1, 0, 1]
        blocos = list(cartesian_product(*[[MAPA_ELEMENTOS['parede'], MAPA_ELEMENTOS['vazio']]] * 4))
        valores_base = np.array([0,0,0,0], dtype=np.float)

        for i in horizontal_comida:
            for j in vertical_comida:
                for esquerda, acima, direita, abaixo in blocos:
                    nova_qtable[self.estado_str(i, j, esquerda, acima, direita, abaixo)] = valores_base.copy()
        
        self.qtable = nova_qtable

    # Verifica se o bloco em questão é passável ou não
    def __bloco_passavel(self, bloco):
        if bloco in (MAPA_ELEMENTOS['corpo'], MAPA_ELEMENTOS['parede']):
            resposta = 1
        else:
            resposta = 2

        return resposta

    # Calcula o direcionamento diante da comida
    def __posicao_comida(self, distancia):
        if distancia < 0:
            resposta = -1
        else:
            resposta = 0
        
        return resposta
        
    # Gera a string que representa o estado do ambiente naquele momeno
    def estado_str(self, horizontal_comida, vertical_comida, bloco_esq, bloco_acima, bloco_dir, bloco_abaixo):
        
        resposta = "(H:{}|V:{}|ESQ:{}|ACI:{}|DIR:{}|ABA:{})".format(
            self.__posicao_comida(horizontal_comida),
            self.__posicao_comida(vertical_comida),
            self.__bloco_passavel(bloco_esq),
            self.__bloco_passavel(bloco_acima),
            self.__bloco_passavel(bloco_dir),
            self.__bloco_passavel(bloco_abaixo)
        )

        return resposta
        
    # Gera a string que representa o estado do ambiente naquele momento
    def fazer_acao(self, posicao_comida, posicao_cobra, cobra_proximidades, posicao_pescoco = None):

        comida_x, comida_y = posicao_comida
        cobra_x, cobra_y = posicao_cobra

        cobra_esq, cobra_aci, cobra_dir, cobra_aba = cobra_proximidades

        distancia_cobra_comida_x = cobra_x - comida_x
        distancia_cobra_comida_y = cobra_y - comida_y

        estado = self.estado_str(distancia_cobra_comida_x, distancia_cobra_comida_y, cobra_esq, cobra_aci, cobra_dir, cobra_aba)
        valores = self.qtable[estado]

        resultado = None
        acao = np.random.choice(['explore', 'exploit'], p=[self.epsilon, 1-self.epsilon], size=1)
        valores_iguais = np.all([valores[0] == valores])

        movimento_valido = list(filter(lambda x: x!=posicao_pescoco, list(ql.acoes.values())))

        if acao == 'explore' or valores_iguais:
            resultado = np.random.choice(movimento_valido)
        else:
            maior_index = np.argmax(valores)
            resultado = self.acoes[maior_index]
        
        return resultado
    
    # Com essa função conseguimos pegar um histórico de jogadas e atualizar a tabela
    def atualizar_qtable(self, historia_status, morreu):
        
        historia = historia_status[::-1]
        if morreu:
            recompensa = -100
            acao = historia[0]['movimento']

            comida_x, comida_y = historia[0]['comida']
            cobra_x, cobra_y = historia[0]['cobra']

            cobra_esq, cobra_aci, cobra_dir, cobra_aba = historia[0]['cobra_proximidades']

            distancia_cobra_comida_x = cobra_x - comida_x
            distancia_cobra_comida_y = cobra_y - comida_y

            estado = self.estado_str(distancia_cobra_comida_x, distancia_cobra_comida_y, cobra_esq, cobra_aci, cobra_dir, cobra_aba)
            valor_antigo = self.qtable[estado][self.acoes_rev[acao]]
            valor_novo = (1-self.alpha) * valor_antigo + self.alpha * recompensa
            self.qtable[estado][self.acoes_rev[acao]] = valor_novo

            for en, momento in enumerate(historia[1:-1], start=1):

                # Estado atual
                acao = momento['movimento']
                tamanho = momento['tamanho']
                comida_x, comida_y = momento['comida']
                cobra_x, cobra_y = momento['cobra']
                cobra_esq, cobra_aci, cobra_dir, cobra_aba = momento['cobra_proximidades']
                distancia_cobra_comida_x = cobra_x - comida_x
                distancia_cobra_comida_y = cobra_y - comida_y

                # Estado anterior
                momento_anterior = historia[en + 1]
                acao_anterior = momento_anterior['movimento']
                tamanho_anterior = momento_anterior['tamanho']
                comida_x_anterior, comida_y_anterior = momento_anterior['comida']
                cobra_x_anterior, cobra_y_anterior = momento_anterior['cobra']
                cobra_esq_anterior, cobra_aci_anterior, cobra_dir_anterior, cobra_aba_anterior = momento_anterior['cobra_proximidades']
                distancia_cobra_comida_x_anterior = cobra_x_anterior - comida_x_anterior
                distancia_cobra_comida_y_anterior = cobra_y_anterior - comida_y_anterior

                # Recompensa
                distancia_piorou = np.abs(distancia_cobra_comida_x) > np.abs(distancia_cobra_comida_x_anterior)
                distancia_piorou = distancia_piorou or np.abs(distancia_cobra_comida_y) > np.abs(distancia_cobra_comida_y_anterior)

                if tamanho > tamanho_anterior:
                    recompensa = 100
                elif distancia_piorou:
                    recompensa = -90
                else:
                    recompensa = 90

                estado_atual = self.estado_str(
                    distancia_cobra_comida_x, 
                    distancia_cobra_comida_y, 
                    cobra_esq, 
                    cobra_aci, 
                    cobra_dir, 
                    cobra_aba
                )
                estado_anterior = self.estado_str(
                   distancia_cobra_comida_x_anterior, 
                    distancia_cobra_comida_y_anterior, 
                    cobra_esq_anterior, 
                    cobra_aci_anterior, 
                    cobra_dir_anterior, 
                    cobra_aba_anterior 
                )

                valor_anterior = self.qtable[estado_anterior][self.acoes_rev[acao_anterior]]
                valor_atual = self.qtable[estado_atual]

                valor_anterior_novo = ((1-self.alpha)*(valor_anterior)) + (self.alpha * (recompensa + (self.gamma*valor_atual.max())))
                self.qtable[estado_anterior][self.acoes_rev[acao_anterior]] = valor_anterior_novo    

## Configurações do Jogo - Parte 2

Vamos criar aqui 2 funções, uma para salvar e outra para ler Qtable em arquivos.

### Salvar Q-Table

In [11]:
def save_qtable(QL, filename='../data/qtable.pk'):
    with open(filename, 'wb') as f:
        pickle.dump(QL, f)

### Ler Q-Table

In [12]:
def read_qtable(filename='../data/qtable.pk'):
    res = None
    with open(filename, 'rb') as f:
        res = pickle.load(f)
    
    return res

## Gerar Animação da História

Com a função abaixo, vamos conseguir gerar uma sequência de matrizes como uma história animada e transforma-lá em vídeo.

In [13]:
%matplotlib notebook
def gerar_historia_status(historia_status, mapa_cores=MAPA_CORES, duration_frame= 100):
    historia = []
    frames = 0
    for cada_status in historia_status:
        historia.append(substituir_cores(cada_status['mapa'], mapa_cores))
        frames += 1
    
    x, y = historia_status[0]['mapa'].shape[::-1]
    figura = plt.figure(figsize=np.array([x, y])/5)
    plt.axis('off');
    primeira_cena = historia[0]
    imagem = plt.imshow(primeira_cena, aspect='auto');

    def funcao_animacao(i):
        imagem.set_array(historia[i])
        return [imagem]
    animacao = animation.FuncAnimation(
        figura, 
        funcao_animacao, 
        frames = frames,
        interval = duration_frame, # in ms
    );

    return animacao


### Jogos aleatórios

A função abaixo, faz com que o jogo tenha estados iniciais diferentes.

In [14]:
def jogos_aleatorios():
    cobra_size = 1
    if np.random.choice([False, True], p=[0.7, 0.3]):
        cobra_size = np.random.randint(2, 12)
    
    return cobra_size

## O Jogo

### Treinamento